In [73]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import os

In [74]:
# Set random seed for reproducibility
np.random.seed(42)

In [75]:
# 1. Load Data
# TfL data: separate files for each year (2019–2023)
tfl_files = {
    2019: "raw_data/Journeys_2019.csv",
    2020: "raw_data/Journeys_2020.csv",
    2021: "raw_data/Journeys_2021.csv",
    2022: "raw_data/Journeys_2022.csv",
    2023: "raw_data/Journeys_2023.csv"
}

# Initialize empty list to store TfL data
tfl_dfs = []

# Load and concatenate TfL files
for year, file in tfl_files.items():
    if os.path.exists(file):
        df = pd.read_csv(file)
        # Sum TubeJourneyCount and BusJourneyCount into total passenger_count
        df['passenger_count'] = df['TubeJourneyCount'] + df['BusJourneyCount']
        # Keep only TravelDate and passenger_count
        df = df[['TravelDate', 'passenger_count']]
        tfl_dfs.append(df)
    else:
        print(f"Warning: File {file} not found. Skipping...")

# Combine TfL data
if tfl_dfs:
    tfl_data = pd.concat(tfl_dfs, ignore_index=True)
else:
    raise FileNotFoundError("No TfL data files were found.")

# Load weather data
weather_data = pd.read_csv("raw_data/london_weather_data_1979_to_2023.csv")

In [76]:
# 2. Data Cleaning (TfL Data)
# Convert date to datetime
tfl_data['TravelDate'] = pd.to_datetime(tfl_data['TravelDate'], format='%Y%m%d', errors='coerce')

# Check for invalid dates
print(f"TfL data: {len(tfl_data)} rows before dropping NaT")
tfl_data = tfl_data.dropna(subset=['TravelDate'])
print(f"TfL data: {len(tfl_data)} rows after dropping NaT")

# Handle missing passenger counts
tfl_data['passenger_count'] = tfl_data['passenger_count'].fillna(tfl_data.groupby(tfl_data['TravelDate'].dt.month)['passenger_count'].transform('mean'))

# Remove outliers (z-score > 3)
z_scores = np.abs((tfl_data['passenger_count'] - tfl_data['passenger_count'].mean()) / tfl_data['passenger_count'].std())
tfl_data = tfl_data[z_scores < 3]
print(f"TfL data after outlier removal: {len(tfl_data)} rows")


TfL data: 1826 rows before dropping NaT
TfL data: 1826 rows after dropping NaT
TfL data after outlier removal: 1826 rows


In [77]:
# 3. Data Cleaning (Weather Data)
# Rename columns to match naming convention
weather_data = weather_data.rename(columns={
    'DATE': 'date',
    'TX': 'max_temp',
    'TN': 'min_temp',
    'TG': 'mean_temp',
    'SS': 'sunshine',
    'SD': 'snow_depth',
    'RR': 'precipitation',
    'QQ': 'global_radiation',
    'PP': 'pressure',
    'HU': 'humidity',
    'CC': 'cloud_cover'
})

# Convert date to datetime
weather_data['date'] = pd.to_datetime(weather_data['date'], format='%Y%m%d', errors='coerce')

# Check for invalid dates
print(f"Weather data: {len(weather_data)} rows before dropping NaT")
weather_data = weather_data.dropna(subset=['date'])
print(f"Weather data: {len(weather_data)} rows after dropping NaT")

# Handle missing weather values
weather_columns = ['max_temp', 'min_temp', 'mean_temp', 'sunshine', 'snow_depth', 'precipitation',
                   'global_radiation', 'pressure', 'humidity', 'cloud_cover']
for col in weather_columns:
    weather_data[col] = weather_data[col].fillna(weather_data.groupby(weather_data['date'].dt.month)[col].transform('mean'))

# Convert units
weather_data['max_temp'] = weather_data['max_temp'] / 10
weather_data['min_temp'] = weather_data['min_temp'] / 10
weather_data['mean_temp'] = weather_data['mean_temp'] / 10
weather_data['sunshine'] = weather_data['sunshine'] / 10
weather_data['precipitation'] = weather_data['precipitation'] / 10
weather_data['pressure'] = weather_data['pressure'] / 10

# Remove invalid entries
weather_data = weather_data[weather_data['precipitation'] >= 0]
print(f"Weather data after cleaning: {len(weather_data)} rows")

Weather data: 16436 rows before dropping NaT
Weather data: 16436 rows after dropping NaT
Weather data after cleaning: 16436 rows


In [78]:
# 4. Temporal Alignment
# Aggregate TfL data to daily total
tfl_data = tfl_data.rename(columns={'TravelDate': 'date'})
daily_tfl = tfl_data.groupby('date')['passenger_count'].sum().reset_index()

# Merge with weather data
merged_data = pd.merge(daily_tfl, weather_data, on='date', how='inner')
print(f"Merged data: {len(merged_data)} rows")

# Filter for 2019–2022 (training) and 2023 (testing)
train_data = merged_data[merged_data['date'].dt.year.isin([2019, 2020, 2021, 2022])]
test_data = merged_data[merged_data['date'].dt.year == 2023]
print(f"Train data (2019–2022): {len(train_data)} rows")
print(f"Test data (2023): {len(test_data)} rows")

# Check if train_data is empty
if len(train_data) == 0:
    raise ValueError("Train data is empty. Check date ranges in TfL and weather data for overlap in 2019–2022.")

Merged data: 1826 rows
Train data (2019–2022): 1461 rows
Test data (2023): 365 rows


In [79]:
# 5. Feature Engineering
# Ensure date is datetime
train_data['date'] = pd.to_datetime(train_data['date'])
test_data['date'] = pd.to_datetime(test_data['date'])

# Add temporal features for train_data
train_data['day_of_week'] = train_data['date'].dt.dayofweek
train_data['month'] = train_data['date'].dt.month
train_data['season'] = train_data['month'].map({
    1: 'Winter', 2: 'Winter', 3: 'Spring', 4: 'Spring', 5: 'Spring',
    6: 'Summer', 7: 'Summer', 8: 'Summer', 9: 'Autumn', 10: 'Autumn',
    11: 'Autumn', 12: 'Winter'
})
train_data['is_raining'] = (train_data['precipitation'] > 0).astype(int)
train_data['passenger_count_lag1'] = train_data['passenger_count'].shift(1)
train_data['temp_7d_mean'] = train_data['mean_temp'].rolling(window=7, min_periods=1).mean()

# Add temporal features for test_data (skip if empty)
if len(test_data) > 0:
    test_data['day_of_week'] = test_data['date'].dt.dayofweek
    test_data['month'] = test_data['date'].dt.month
    test_data['season'] = test_data['month'].map({
        1: 'Winter', 2: 'Winter', 3: 'Spring', 4: 'Spring', 5: 'Spring',
        6: 'Summer', 7: 'Summer', 8: 'Summer', 9: 'Autumn', 10: 'Autumn',
        11: 'Autumn', 12: 'Winter'
    })
    test_data['is_raining'] = (test_data['precipitation'] > 0).astype(int)
    test_data['passenger_count_lag1'] = test_data['passenger_count'].shift(1)
    test_data['temp_7d_mean'] = test_data['mean_temp'].rolling(window=7, min_periods=1).mean()

# Handle missing values from lag/rolling features
train_data = train_data.fillna(method='bfill')
if len(test_data) > 0:
    test_data = test_data.fillna(method='bfill')

C:\Users\lenovo\AppData\Local\Temp\ipykernel_10068\3666198065.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['date'] = pd.to_datetime(train_data['date'])
C:\Users\lenovo\AppData\Local\Temp\ipykernel_10068\3666198065.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['date'] = pd.to_datetime(test_data['date'])
C:\Users\lenovo\AppData\Local\Temp\ipykernel_10068\3666198065.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc

In [80]:
# 6. Handle Redundancies
# Drop highly correlated weather features
corr_matrix = train_data[weather_columns].corr()
high_corr = [(col1, col2) for col1 in corr_matrix.columns for col2 in corr_matrix.index
             if col1 < col2 and abs(corr_matrix.loc[col2, col1]) > 0.8]
if high_corr:
    drop_cols = [col2 for col1, col2 in high_corr]
    print(f"Dropped columns due to high correlation: {drop_cols}")
    train_data = train_data.drop(columns=drop_cols)
    if len(test_data) > 0:
        test_data = test_data.drop(columns=drop_cols)


Dropped columns due to high correlation: ['mean_temp', 'min_temp', 'sunshine']


In [81]:
# 7. Normalize Numerical Features
# Dynamically select numerical columns that exist
base_numerical_cols = ['passenger_count', 'max_temp', 'min_temp', 'mean_temp', 'precipitation',
                       'sunshine', 'snow_depth', 'global_radiation', 'pressure', 'humidity',
                       'cloud_cover', 'passenger_count_lag1', 'temp_7d_mean']
numerical_cols = [col for col in base_numerical_cols if col in train_data.columns]
print(f"Numerical columns for normalization: {numerical_cols}")

# Apply normalization only if train_data is not empty
if len(train_data) > 0:
    scaler = MinMaxScaler()
    train_data[numerical_cols] = scaler.fit_transform(train_data[numerical_cols])
    if len(test_data) > 0:
        test_data[numerical_cols] = scaler.transform(test_data[numerical_cols])
else:
    raise ValueError("Cannot normalize: train_data is empty.")

Numerical columns for normalization: ['passenger_count', 'max_temp', 'precipitation', 'snow_depth', 'global_radiation', 'pressure', 'humidity', 'cloud_cover', 'passenger_count_lag1', 'temp_7d_mean']


In [82]:
# 8. Save Processed Data
train_data.to_csv('processed_train_data.csv', index=False)
if len(test_data) > 0:
    test_data.to_csv('processed_test_data.csv', index=False)
else:
    print("Warning: test_data is empty, skipping save of test_data CSV.")

In [83]:
# Output confirmation
print("Processed data saved as 'processed_train_data.csv'")
if len(test_data) > 0:
    print("Processed data saved as 'processed_test_data.csv'")

Processed data saved as 'processed_train_data.csv'
Processed data saved as 'processed_test_data.csv'
